In [1]:
from src.diametrics import transform

In [2]:
dexcom_data = transform.transform_directory(directory='tests/test_data/dexcom/', device='dexcom')
dexcom_data.head()

,time,glc,ID
0,2023-03-08 00:04:00,6.4,dexcom_eur_01
1,2023-03-08 00:09:00,6.5,dexcom_eur_01
2,2023-03-08 00:13:59,6.1,dexcom_eur_01
3,2023-03-08 00:18:59,6.5,dexcom_eur_01
4,2023-03-08 00:23:59,6.3,dexcom_eur_01


In [9]:
df = dexcom_data

In [12]:
df = df.sort_values(['ID', 'time'], 
                        ascending=True).reset_index(drop=True)
# Group by 'ID' and calculate time difference
df['time_diff'] = df.groupby('ID')['time'].diff()

# Convert time difference to minutes
df['time_diff'] = df['time_diff'].dt.total_seconds() / 60

# Identify where the time difference is greater than 16 minutes
df['missing_reading'] = df['time_diff'] > 6

# Filter to show only the rows where readings are missing
missing_readings_df = df[df['missing_reading']]

# Get indices of missing readings
missing_indices = df[df['missing_reading']].index

# Get indices of rows just before the missing readings
previous_indices = missing_indices - 1

# Combine indices and remove duplicates
combined_indices = missing_indices.union(previous_indices).drop_duplicates()

# Filter the original DataFrame for these indices
combined_df = df.loc[combined_indices]

In [13]:
combined_df

,time,glc,ID,time_diff,missing_reading
500,2023-03-09 17:44:03,15.6,dexcom_eur_01,4.983333,False
501,2023-03-09 21:39:04,10.9,dexcom_eur_01,235.016667,True
2832,2023-03-17 23:54:21,11.9,dexcom_eur_01,5.000000,False
2833,2023-03-18 00:24:21,12.3,dexcom_eur_01,30.000000,True
3351,2023-03-19 19:34:25,9.3,dexcom_eur_01,5.000000,False
...,...,...,...,...,...
10693,2023-03-18 15:31:54,10.3,dexcom_eur_03,5.000000,False
10694,2023-03-18 15:41:55,12,dexcom_eur_03,10.016667,True
10695,2023-03-18 15:56:55,15.9,dexcom_eur_03,15.000000,True
11034,2023-03-19 20:11:49,8.7,dexcom_eur_03,5.000000,False
